In [1]:
#MODELO XGBOOST optimizado y sin sopbreajuste
#==========================================================================================

import pandas as pd
import xgboost as xgb
import joblib
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Cargar el dataset
df = pd.read_csv('data/data_FINAL2.csv')

# 2. Convertir la variable objetivo "Precio" a numérico y eliminar registros sin precio
df['Precio'] = pd.to_numeric(df['Precio'], errors='coerce')
df = df.dropna(subset=['Precio'])

# 3. Seleccionar las variables predictoras y la variable objetivo
y = df['Precio']
features = ['Marca', 'Modelo', 'Provincia', 'Año', 'Kilometraje', 
            'Transmisión', 'Dirección', 'Motor', 'Tracción', 'Color', 'Combustible']
X = df[features]

# 4. Aplicar Label Encoding a las columnas categóricas y guardar los encoders
categorical_cols = ['Marca', 'Modelo', 'Provincia', 'Transmisión', 'Dirección', 'Tracción', 'Color', 'Combustible']
X_encoded = X.copy()
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    encoders[col] = le  # Guardamos el encoder

# Guardar los encoders en un archivo para usarlos en la predicción
with open('encoders/encoders_xgboost2.pkl', 'wb') as f:
    pickle.dump(encoders, f)

# 5. Dividir el dataset en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.15, random_state=42)

best_xgb = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.8,  # Aumentamos el número de variables usadas en cada árbol
    gamma=0.1,  # Permitimos más divisiones
    learning_rate=0.025,  # Reducimos la tasa de aprendizaje para mejorar precisión
    max_depth=4,  # Árboles más profundos para capturar más patrones
    n_estimators=700,  # Más árboles para mejorar aprendizaje
    subsample=0.8,  # Usamos más datos por cada árbol
    alpha=0.5,  # Reducimos la regularización L1 para permitir más flexibilidad
    lambda_=1.0,  # Reducimos la regularización L2 para evitar que los pesos sean demasiado pequeños
    random_state=42,
    n_jobs=-1
)



# 7. Entrenar el modelo
best_xgb.fit(X_train, y_train)

# 8. Evaluación del modelo
y_train_pred = best_xgb.predict(X_train)
y_test_pred = best_xgb.predict(X_test)

# Calcular métricas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)

mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

# 9. Calcular la varianza entre entrenamiento y prueba
varianza_mse = mse_train - mse_test
varianza_mae = mae_train - mae_test
varianza_r2 = r2_train - r2_test

# 10. Mostrar resultados
print("\n======== RESULTADOS ========\n")
print("🔹 **Entrenamiento:**")
print(f" - MSE: {mse_train:.2f}")
print(f" - MAE: {mae_train:.2f}")
print(f" - RMSE: {rmse_train:.2f}")
print(f" - R²: {r2_train:.4f}")

print("\n🔹 **Prueba:**")
print(f" - MSE: {mse_test:.2f}")
print(f" - MAE: {mae_test:.2f}")
print(f" - RMSE: {rmse_test:.2f}")
print(f" - R²: {r2_test:.4f}")

print("\n🔹 **Varianza (Diferencia entre entrenamiento y prueba):**")
print(f" - MSE Varianza: {varianza_mse:.2f}")
print(f" - MAE Varianza: {varianza_mae:.2f}")
print(f" - RMSE Varianza: {rmse_train - rmse_test:.2f}")
print(f" - R² Varianza: {varianza_r2:.4f}")

# 11. Detectar sobreajuste
if abs(rmse_train - rmse_test) > 1000 or abs(varianza_r2) > 0.1:
    print("\n **Posible sobreajuste detectado** ⚠️")
    print("El modelo tiene un error mucho menor en entrenamiento que en prueba.")
else:
    print("\n **El modelo generaliza bien**")

# Guardar el modelo entrenado
joblib.dump(best_xgb, 'models/modelo_xgboost_optimizado.pkl')
print("\n El modelo optimizado se ha guardado en 'models/modelo_xgboost_optimizado.pkl'")


d:\mineria_auto\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [22:51:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "lambda_" } are not used.

  warnings.warn(smsg, UserWarning)



======== RESULTADOS ========

🔹 **Entrenamiento:**
 - MSE: 11540089.22
 - MAE: 2403.70
 - RMSE: 3397.07
 - R²: 0.8877

🔹 **Prueba:**
 - MSE: 17067478.20
 - MAE: 2854.61
 - RMSE: 4131.28
 - R²: 0.8179

🔹 **Varianza (Diferencia entre entrenamiento y prueba):**
 - MSE Varianza: -5527388.98
 - MAE Varianza: -450.91
 - RMSE Varianza: -734.21
 - R² Varianza: 0.0698

 **El modelo generaliza bien**

 El modelo optimizado se ha guardado en 'models/modelo_xgboost_optimizado.pkl'
